In [1]:
import cv2 as cv
import math
import time
import argparse

def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes


# parser = argparse.ArgumentParser(description='Use this script to run age and gender recognition using OpenCV.')
# parser.add_argument('--input', help='Path to input image or video file. Skip this argument to capture frames from a camera.')
# parser.add_argument("--device", default="cpu", help="Device to inference on")

# args 


args={'device':"gpu",'input': 0}

faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"

ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"

genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(2-6)', '(8-12)', '(12-19)', '(20-27)','(28-34)', '(35-43)', '(44-60)', '(61-100)']
genderList = ['Male', 'Female']

# Load network
ageNet = cv.dnn.readNet(ageModel, ageProto)
genderNet = cv.dnn.readNet(genderModel, genderProto)
faceNet = cv.dnn.readNet(faceModel, faceProto)


if args['device'] == "cpu":
    ageNet.setPreferableBackend(cv.dnn.DNN_TARGET_CPU)

    genderNet.setPreferableBackend(cv.dnn.DNN_TARGET_CPU)
    
    faceNet.setPreferableBackend(cv.dnn.DNN_TARGET_CPU)

    print("Using CPU device")
elif args['device'] == "gpu":
    ageNet.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
    ageNet.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA)

    genderNet.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
    genderNet.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA)

    genderNet.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
    genderNet.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA)
    print("Using GPU device")


# Open a video file or an image file or a camera stream
cap = cv.VideoCapture(args['input'] if args['input'] else 0)
padding = 20
while cv.waitKey(1)<0:
    # Read frame
    t = time.time()
    hasFrame, frame = cap.read()
    if not hasFrame:
        cv.waitKey()
        break

    frameFace, bboxes = getFaceBox(faceNet, frame)
    if not bboxes:
        print("No face Detected, Checking next frame")
        continue
   
    
    for bbox in bboxes:
        # print(bbox)
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        blob = cv.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        # print("Gender Output : {}".format(genderPreds))
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print("Age Output : {}".format(agePreds))
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv.putText(frameFace, label, (bbox[0], bbox[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv.LINE_AA)
        cv.imshow("Age Gender Demo", frameFace)
        # cv.imwrite("age-gender-out-{}".format(args.input),frameFace)
#         if cv.waitKey(1) & 0xFF == ord('q'):
#             break
    print("time : {:.3f}".format(time.time() - t))
        

# Release handle to the webcam
# video_capture.release()
cv.destroyAllWindows()


Using GPU device
Gender : Male, conf = 1.000
Age Output : [[1.3006789e-03 2.4201032e-02 4.5893848e-01 5.0928718e-01 5.9124930e-03
  2.6330675e-04 6.3345433e-05 3.3434313e-05]]
Age : (12-19), conf = 0.509
time : 1.914
Gender : Male, conf = 1.000
Age Output : [[3.2353602e-04 1.1145992e-02 5.5244362e-01 4.2781317e-01 8.0142329e-03
  2.0680529e-04 3.6232723e-05 1.6439519e-05]]
Age : (8-12), conf = 0.552
time : 0.041
Gender : Male, conf = 1.000
Age Output : [[1.19598955e-02 7.03909770e-02 8.02585065e-01 9.46497321e-02
  1.93880070e-02 7.12266134e-04 1.95674700e-04 1.18381569e-04]]
Age : (8-12), conf = 0.803
time : 0.041
Gender : Male, conf = 1.000
Age Output : [[1.50983594e-03 3.04061398e-02 2.41418630e-01 7.19896972e-01
  6.17797440e-03 4.42585006e-04 1.11091176e-04 3.68345754e-05]]
Age : (12-19), conf = 0.720
time : 0.046
Gender : Male, conf = 1.000
Age Output : [[1.50983594e-03 3.04061398e-02 2.41418630e-01 7.19896972e-01
  6.17797440e-03 4.42585006e-04 1.11091176e-04 3.68345754e-05]]
Ag

time : 0.025
Gender : Male, conf = 1.000
Age Output : [[5.5034677e-03 3.7134293e-02 1.3741761e-01 8.0396289e-01 9.7701708e-03
  5.0149276e-03 9.0836885e-04 2.8830249e-04]]
Age : (12-19), conf = 0.804
time : 0.028
Gender : Male, conf = 1.000
Age Output : [[3.0197566e-03 2.9600672e-02 1.1234077e-01 8.4401309e-01 6.3792006e-03
  3.8468256e-03 5.8662461e-04 2.1314129e-04]]
Age : (12-19), conf = 0.844
time : 0.052
Gender : Male, conf = 1.000
Age Output : [[3.0197566e-03 2.9600672e-02 1.1234077e-01 8.4401309e-01 6.3792006e-03
  3.8468256e-03 5.8662461e-04 2.1314129e-04]]
Age : (12-19), conf = 0.844
time : 0.023
Gender : Male, conf = 1.000
Age Output : [[2.0632595e-03 2.1950560e-02 5.5645663e-02 9.1340077e-01 4.3304828e-03
  2.1277170e-03 3.3078372e-04 1.5082485e-04]]
Age : (12-19), conf = 0.913
time : 0.039
Gender : Male, conf = 1.000
Age Output : [[9.9351804e-04 1.3804498e-02 6.3938178e-02 9.1687632e-01 2.6158411e-03
  1.4639244e-03 2.2300829e-04 8.4613494e-05]]
Age : (12-19), conf = 0.917


Gender : Male, conf = 1.000
Age Output : [[0.04606317 0.19389468 0.18166153 0.544494   0.02124094 0.01038927
  0.00159425 0.00066216]]
Age : (12-19), conf = 0.544
time : 0.073
Gender : Male, conf = 1.000
Age Output : [[0.0713793  0.19164528 0.14947806 0.5286579  0.03854754 0.01603715
  0.00313408 0.0011207 ]]
Age : (12-19), conf = 0.529
time : 0.045
Gender : Male, conf = 1.000
Age Output : [[0.06726097 0.18956836 0.11765178 0.58801204 0.0230546  0.01145269
  0.00220603 0.00079353]]
Age : (12-19), conf = 0.588
time : 0.017
Gender : Male, conf = 1.000
Age Output : [[2.2909777e-02 1.4259338e-01 2.3684613e-01 5.7089514e-01 1.8360890e-02
  6.9221579e-03 1.0302006e-03 4.4235383e-04]]
Age : (12-19), conf = 0.571
time : 0.105
Gender : Male, conf = 1.000
Age Output : [[0.04937255 0.16963811 0.09800945 0.6497086  0.02219777 0.00860746
  0.00177082 0.00069514]]
Age : (12-19), conf = 0.650
time : 0.025
Gender : Male, conf = 1.000
Age Output : [[0.12016035 0.18619081 0.05192706 0.6014801  0.0251521

Gender : Male, conf = 1.000
Age Output : [[0.03739173 0.11068061 0.05673746 0.7246255  0.03804577 0.02268588
  0.00808252 0.00175045]]
Age : (12-19), conf = 0.725
time : 0.041
Gender : Male, conf = 1.000
Age Output : [[0.02234129 0.06950227 0.03504269 0.84050333 0.01729006 0.01073269
  0.00365396 0.00093368]]
Age : (12-19), conf = 0.841
time : 0.042
Gender : Male, conf = 1.000
Age Output : [[6.7357635e-03 3.6045916e-02 4.5680147e-02 8.9721107e-01 7.5507946e-03
  4.6024970e-03 1.7481399e-03 4.2571942e-04]]
Age : (12-19), conf = 0.897
time : 0.041
Gender : Male, conf = 1.000
Age Output : [[6.7357635e-03 3.6045916e-02 4.5680147e-02 8.9721107e-01 7.5507946e-03
  4.6024970e-03 1.7481399e-03 4.2571942e-04]]
Age : (12-19), conf = 0.897
time : 0.046
Gender : Male, conf = 1.000
Age Output : [[0.0160548  0.08526748 0.05895447 0.81080115 0.01464006 0.01017255
  0.00315273 0.00095671]]
Age : (12-19), conf = 0.811
time : 0.044
Gender : Male, conf = 1.000
Age Output : [[5.2011460e-03 3.7154701e-02 3

time : 0.041
Gender : Male, conf = 1.000
Age Output : [[5.7658004e-03 3.0853646e-02 6.9750123e-02 8.5080206e-01 2.7568310e-02
  1.2323963e-02 2.1874732e-03 7.4864359e-04]]
Age : (12-19), conf = 0.851
time : 0.041
Gender : Male, conf = 1.000
Age Output : [[5.9505454e-03 3.6571555e-02 5.8662631e-02 8.6808121e-01 1.8444454e-02
  9.7483611e-03 1.9697589e-03 5.7158747e-04]]
Age : (12-19), conf = 0.868
time : 0.043
Gender : Male, conf = 1.000
Age Output : [[5.9505454e-03 3.6571555e-02 5.8662631e-02 8.6808121e-01 1.8444454e-02
  9.7483611e-03 1.9697589e-03 5.7158747e-04]]
Age : (12-19), conf = 0.868
time : 0.035
Gender : Male, conf = 1.000
Age Output : [[0.04290617 0.13051276 0.09242105 0.64386076 0.05933973 0.0224267
  0.00641293 0.00211995]]
Age : (12-19), conf = 0.644
time : 0.042
Gender : Male, conf = 1.000
Age Output : [[0.02030538 0.07495401 0.08675818 0.7423185  0.04934723 0.02073613
  0.00413082 0.00144974]]
Age : (12-19), conf = 0.742
time : 0.036
Gender : Male, conf = 1.000
Age Outp

time : 0.037
Gender : Male, conf = 1.000
Age Output : [[2.0185416e-03 1.3626163e-02 4.2879529e-02 9.0971220e-01 2.0973975e-02
  8.9230044e-03 1.4096019e-03 4.5698963e-04]]
Age : (12-19), conf = 0.910
time : 0.035
Gender : Male, conf = 1.000
Age Output : [[2.4045431e-03 1.7571747e-02 4.8179094e-02 9.0866715e-01 1.4778817e-02
  6.5719341e-03 1.4197761e-03 4.0691256e-04]]
Age : (12-19), conf = 0.909
time : 0.036
Gender : Male, conf = 1.000
Age Output : [[1.7416970e-03 1.4385399e-02 4.5028023e-02 9.1720355e-01 1.4124515e-02
  6.0505592e-03 1.1406571e-03 3.2565166e-04]]
Age : (12-19), conf = 0.917
time : 0.057
Gender : Male, conf = 1.000
Age Output : [[1.7416970e-03 1.4385399e-02 4.5028023e-02 9.1720355e-01 1.4124515e-02
  6.0505592e-03 1.1406571e-03 3.2565166e-04]]
Age : (12-19), conf = 0.917
time : 0.037
Gender : Male, conf = 1.000
Age Output : [[0.01592785 0.06869425 0.06321429 0.79321194 0.0354921  0.01781028
  0.00426647 0.00138282]]
Age : (12-19), conf = 0.793
time : 0.035
Gender : Ma

KeyboardInterrupt: 

In [2]:
%tb

SystemExit: 2